<a href="https://colab.research.google.com/github/lszam/gds-ssebrazil/blob/main/grav_tratar_dados_do_ibge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Gravimetria - Ajustar dados do IBGE para concatenação como outros dados

In [2]:
import zipfile
import os
import geopandas as gpd
import pandas as pd
from google.colab import files

# === 1. FAZER UPLOAD DO ARQUIVO ZIP ORIGINAL ===
uploaded = files.upload()

Saving ibge_25fev2025.zip to ibge_25fev2025.zip


In [3]:
# === 2. EXTRAIR OS ARQUIVOS DO ZIP ===

# Obter o nome do arquivo ZIP
zip_filename = list(uploaded.keys())[0]

# Criar pasta temporária para extração
extract_path = "/content/shapefile_extracted"
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [8]:
# Identificar o arquivo .shp dentro da pasta extraída
shapefile_path = None
for file in os.listdir(extract_path):
    if file.endswith(".shp"):
        shapefile_path = os.path.join(extract_path, file)
        break

if shapefile_path is None:
    raise ValueError("😕 Nenhum arquivo .shp encontrado no ZIP!")
else:
    print("✅ Arquivo .shp encontrado!")

✅ Arquivo .shp encontrado!


In [5]:
# === 3. CARREGAR O SHAPEFILE ===
gdf = gpd.read_file(shapefile_path)

# === 4. SELECIONAR COLUNAS DESEJADAS ===
columns_to_keep = {
    "LATITUDE": "LAT",
    "LONGITUDE": "LON",
    "ALTGEOM": "ALT_GEOM",
    "ALTO_ORTO": "ALT_ORTO",
    "ALT_NORMAL": "ALT_NORM",
    "ESTACAO": "ESTACAO",
    "DATUM_ALT": "DAT_ALT",
    "FONT_ALT": "FNT_ALT",
    "MEDICAOALT": "MED_ALT",
    "CALCULOALT": "CAL_ALT",
    "GRAVIDADE": "GRAV",
    "MEDICAOGRA": "MED_GRA",
    "CALCULOGRA": "CAL_GRA",
    "DATUM_GRA": "DAT_GRA"
}

# Filtrar colunas existentes no shapefile e renomeá-las
columns_present = {k: v for k, v in columns_to_keep.items() if k in gdf.columns}
gdf_selected = gdf[list(columns_present.keys())].rename(columns=columns_present)

In [6]:
# === 5. GARANTIR QUE OS VALORES NUMÉRICOS ESTEJAM NO FORMATO CORRETO ===
for col in gdf_selected.select_dtypes(include=['number']).columns:
    gdf_selected[col] = gdf_selected[col].astype(float)

# === 6. SALVAR O NOVO SHAPEFILE ===
output_folder = "/content/shapefile_processed"
os.makedirs(output_folder, exist_ok=True)
output_shapefile_path = os.path.join(output_folder, "BDG_EGPoint_fixed.shp")

# Criar GeoDataFrame com geometria original
gdf_selected = gpd.GeoDataFrame(gdf_selected, geometry=gdf.geometry)

# Salvar no formato ESRI Shapefile com codificação compatível
gdf_selected.to_file(output_shapefile_path, driver="ESRI Shapefile", encoding="latin1")

# === 7. COMPACTAR O SHAPEFILE PARA DOWNLOAD ===
output_zip_path = "/content/BDG_EGPoint_fixed.zip"
with zipfile.ZipFile(output_zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for ext in [".shp", ".shx", ".dbf", ".prj", ".cpg"]:
        file_path = os.path.join(output_folder, "BDG_EGPoint_fixed" + ext)
        if os.path.exists(file_path):
            zipf.write(file_path, os.path.basename(file_path))

# Fazer o download do novo shapefile compactado
files.download(output_zip_path)

print("✅ Processamento concluído! O arquivo ZIP está pronto para download.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Processamento concluído! O arquivo ZIP está pronto para download.
